In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# Membaca file CSV
file_path = 'data beras jawa barat.csv'
df = pd.read_csv(file_path)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2


In [ ]:
# Membaca file CSV dengan titik koma sebagai pemisah
df = pd.read_csv(file_path, delimiter=';')

# Menampilkan 5 baris pertama dari data yang sudah dibaca
df.head()

In [4]:
# Mengubah data dari format lebar ke format panjang
df_long = df.melt(id_vars=["Pasar"], 
                  var_name="Tanggal", 
                  value_name="Harga")

# Mengubah kolom Tanggal menjadi tipe datetime
df_long["Tanggal"] = pd.to_datetime(df_long["Tanggal"], format='%m/%d/%Y')

# Mengubah kolom Harga menjadi tipe numerik
df_long["Harga"] = pd.to_numeric(df_long["Harga"], errors='coerce')

# Menampilkan 5 baris pertama dari data yang sudah diubah
df_long.head()

ValueError: time data '1-Jan' does not match format '%m/%d/%Y' (match)

In [ ]:
# Mengelompokkan data berdasarkan tanggal dan menghitung rata-rata harga
df_long_grouped = df_long.groupby('Tanggal')['Harga'].mean()

# Fungsi untuk mengisi nilai NaN dengan rata-rata harga pada hari itu
def isi_na_dengan_rata_rata(row):
    if pd.isnull(row['Harga']):
        return df_long_grouped[row['Tanggal']]
    return row['Harga']

# Mengisi nilai NaN dalam kolom Harga dengan rata-rata harga pada hari itu
df_long['Harga'] = df_long.apply(isi_na_dengan_rata_rata, axis=1)

# Menghapus baris yang masih berisi NaN setelah pengisian nilai NaN
df_long.dropna(inplace=True)



In [ ]:
# Menggunakan data yang sudah diubah ke format panjang
X = df_long[['Tanggal']]
y = df_long['Harga']

# Mengisi nilai NaN dalam kolom Harga dengan rata-rata harga pada hari itu
df_long['Harga'] = df_long.apply(isi_na_dengan_rata_rata, axis=1)

# Menghapus baris yang masih berisi NaN setelah pengisian nilai NaN
df_long.dropna(inplace=True)

# Membagi data menjadi data pelatihan dan data pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
print("Jumlah nilai NaN dalam kolom Harga:", y_train.isna().sum())

In [ ]:
# # Lanjutkan dengan pelatihan model
# X = df_long[['Tanggal']]
# y = df_long['Harga']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan melatih model Random Forest dengan penyetelan parameter
model = RandomForestRegressor(n_estimators=1000, max_depth=10, min_samples_split=5, random_state=0)
model.fit(X_train, y_train)

In [5]:
# Memprediksi harga untuk 1 bulan ke depan
last_date = df_long['Tanggal'].max()
next_month = last_date + pd.DateOffset(months=1)
future_dates = pd.date_range(start=next_month, periods=30, freq='D')
future_df = pd.DataFrame({'Tanggal': future_dates})
future_df['Harga'] = model.predict(future_df[['Tanggal']])

# Menampilkan data prediksi untuk 1 bulan ke depan
print(future_df)


TypeError: can only concatenate str (not "DateOffset") to str

In [31]:
# Menampilkan prediksi harga tertinggi dan terendah untuk 1 bulan ke depan
harga_tertinggi_1_bulan = max(future_df['Harga'])
harga_terendah_1_bulan = min(future_df['Harga'])
print("Prediksi Harga Tertinggi untuk 1 Bulan ke Depan:", harga_tertinggi_1_bulan)
print("Prediksi Harga Terendah untuk 1 Bulan ke Depan:", harga_terendah_1_bulan)

Prediksi Harga Tertinggi untuk 1 Bulan ke Depan: 14098.6353559883
Prediksi Harga Terendah untuk 1 Bulan ke Depan: 14098.6353559883


In [32]:
# Membuat kolom prediksi klasifikasi
threshold = (harga_tertinggi_1_bulan + harga_terendah_1_bulan) / 2
future_df['Prediksi_Klasifikasi'] = np.where(future_df['Harga'] >= threshold, 1, 0)

# Membuat kolom klasifikasi aktual berdasarkan harga rata-rata sebelumnya
average_price = df_long['Harga'].mean()
df_long['Klasifikasi_Aktual'] = np.where(df_long['Harga'] >= average_price, 1, 0)

# Evaluasi kinerja model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score

# Matriks Konfusi
cm = confusion_matrix(df_long['Klasifikasi_Aktual'], np.where(df_long['Harga'] >= threshold, 1, 0))
print("Confusion Matrix:")
print(cm)

# Akurasi
accuracy = accuracy_score(df_long['Klasifikasi_Aktual'], np.where(df_long['Harga'] >= threshold, 1, 0))
print("Akurasi:", accuracy)

# Presisi
precision = precision_score(df_long['Klasifikasi_Aktual'], np.where(df_long['Harga'] >= threshold, 1, 0))
print("Presisi:", precision)

# F1-Score
f1score = f1_score(df_long['Klasifikasi_Aktual'], np.where(df_long['Harga'] >= threshold, 1, 0))
print("F1-Score:", f1score)

Confusion Matrix:
[[3943    0]
 [3363  590]]
Akurasi: 0.5740881458966566
Presisi: 1.0
F1-Score: 0.2597402597402597
